## Поиск квадруплексов

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.9 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO
import re

pattern_plus = re.compile(r"(?:G{3,5}[ATGC]{1,7}){3,}G{3,5}", re.IGNORECASE)
pattern_minus = re.compile(r"(?:C{3,5}[ATGC]{1,7}){3,}C{3,5}", re.IGNORECASE)

input_fasta = "/content/drive/MyDrive/GCF_013753865.1_Amil_v2.1_genomic.fna"
output_bed = "quadruplexes.bed"

results = []

for record in SeqIO.parse(input_fasta, "fasta"):
    seq = str(record.seq)
    for match in pattern_plus.finditer(seq):
        results.append((record.id, match.start(), match.end(), "+"))
    for match in pattern_minus.finditer(seq):
        results.append((record.id, match.start(), match.end(), "-"))

with open(output_bed, "w") as f:
    for chrom, start, end, strand in results:
        f.write(f"{chrom}\t{start}\t{end}\t{strand}\n")

print(f"Найдено квадруплексов: {len(results)}")


Найдено квадруплексов: 17935


## Zhunt

(запуск на сервере)

In [ ]:
gcc zhunt3-alan1.c -lm -o zhunt3

In [ ]:
./zhunt3 12 8 12 GCF_013753865.1_Amil_v2.1_genomic.fna

In [ ]:
import pandas as pd
chunks = pd.read_csv(
     "GCF_013753865.1_Amil_v2.1_genomic.fna.Z-SCORE",
     skiprows=1,
     names=["start", "end", "col3", "col4", "col5", "score", "seq", "pattern"],
     delim_whitespace=True,
     chunksize=100000
 )

 # Фильтрация
 filtered_chunks = []
 for chunk in chunks:
     filtered_chunk = chunk[chunk["score"] > 400]
     filtered_chunks.append(filtered_chunk)

 zhunt_df = pd.concat(filtered_chunks, ignore_index=True)
 gff = pd.read_csv("genomic.gff", sep="\t", header=None, comment="#",
                   names=["seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"])

 chr_df = gff[gff["type"] == "region"][["seqid", "end"]].reset_index(drop=True)
 chr_df["cumulative_end"] = chr_df["end"].cumsum()
 corrected_records = []

 for _, row in zhunt_df.iterrows():
     for chrom_index, chrom_row in chr_df.iterrows():
         if int(row["start"]) <= chrom_row["cumulative_end"]:
             seqid = chrom_row["seqid"]
             prev_end = chr_df.iloc[chrom_index - 1]["cumulative_end"] if chrom_index > 0 else 0
             corrected_records.append({
                 "seqid": seqid,
                 "start": int(row["start"]) - prev_end,
                 "end": int(row["end"]) - prev_end,
                 "seq": row["seq"]
             })
             break

 corrected_zhunt_df = pd.DataFrame(corrected_records)
 bed_df = corrected_zhunt_df.rename(columns={
     'seqid': 'chr',
     'start': 'chromStart',
     'end': 'chromEnd'
 })
 bed_df.to_csv("zdna_gt400.bed", sep="\t", header=False, index=False, columns=["chr", "chromStart", "chromEnd"])


## ZDNABERT

(запуск на сервере)

In [ ]:
pip install torch transformers biopython scipy

mkdir -p model && cd model

wget -O pytorch_model.bin 'https://drive.google.com/uc?export=download&id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
wget -O config.json 'https://drive.google.com/uc?export=download&id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5'
wget -O special_tokens_map.json 'https://drive.google.com/uc?export=download&id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d'
wget -O tokenizer_config.json 'https://drive.google.com/uc?export=download&id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG'
wget -O vocab.txt 'https://drive.google.com/uc?export=download&id=1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0'


In [ ]:
pip install --user gdown

In [ ]:
export PATH=$HOME/.local/bin:$PATH

In [ ]:
cd ~/model
gdown --id 1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx -O pytorch_model.bin

In [ ]:
ls -lh pytorch_model.bin

In [ ]:
cd ~
python run_zdnabert.py

In [ ]:
# скрипт run_zdnabert.py

import torch
from transformers import BertTokenizer, BertForTokenClassification
from Bio import SeqIO
import numpy as np
import scipy.ndimage
import os

def seq2kmer(seq, k):
    return [seq[x:x+k] for x in range(len(seq)+1-k)]

def split_seq(seq, length=512, pad=16):
    return [seq[x:min(x+length, len(seq))] for x in range(0, len(seq), length - pad)]

def stitch_np_seq(np_seqs, pad=16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res, seq])
    return res

print("Загружаем модель ZDNABERT...")
tokenizer = BertTokenizer.from_pretrained("model/")
model = BertForTokenClassification.from_pretrained("model/")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

fasta_path = "GCF_013753865.1_Amil_v2.1_genomic.fna"
output_file = "zdnabert_results.txt"
threshold = 0.25
min_len = 6

out = []

print(f"Запускаем анализ {fasta_path}...")
for record in SeqIO.parse(fasta_path, "fasta"):
    kmer_seq = seq2kmer(str(record.seq).upper(), 6)
    seq_pieces = split_seq(kmer_seq)
    preds = []

    with torch.no_grad():
        for piece in seq_pieces:
            input_ids = torch.LongTensor(tokenizer.encode(' '.join(piece), add_special_tokens=False)).unsqueeze(0).to(device)
            outputs = model(input_ids)[0]
            probs = torch.softmax(outputs, dim=-1)[0][:,1]
            preds.append(probs.cpu().numpy())

    stitched = stitch_np_seq(preds)
    labeled, max_label = scipy.ndimage.label(stitched > threshold)

    out.append(f">{record.id}")
    out.append("start\tend")
    for label in range(1, max_label + 1):
        region = np.where(labeled == label)[0]
        if len(region) >= min_len:
            out.append(f"{region[0]}\t{region[-1]}")

with open(output_file, "w") as f:
    f.write("\n".join(out))

print(f"Результаты в: {output_file}")


## Аннотация

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

In [4]:
annotation_df = pd.read_csv('/content/drive/MyDrive/genomic.gff', sep='\t', comment='#', header=None, names=[
    "seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"
])

In [5]:
chr_seqid = [
    "NC_058066.1", "NC_058067.1", "NC_058068.1", "NC_058069.1", "NC_058070.1",
    "NC_058071.1", "NC_058072.1", "NC_058073.1", "NC_058074.1", "NC_058075.1",
    "NC_058076.1", "NC_058077.1", "NC_058078.1", "NC_058079.1"
]

In [6]:
chr_rename = {
    "NC_058066.1": "1",
    "NC_058067.1": "2",
    "NC_058068.1": "3",
    "NC_058069.1": "4",
    "NC_058070.1": "5",
    "NC_058071.1": "6",
    "NC_058072.1": "7",
    "NC_058073.1": "8",
    "NC_058074.1": "9",
    "NC_058075.1": "10",
    "NC_058076.1": "11",
    "NC_058077.1": "12",
    "NC_058078.1": "13",
    "NC_058079.1": "14"
}


In [7]:
chr_df = annotation_df[(annotation_df['type']=='region') & (annotation_df['seqid'].isin(chr_seqid))]
chr_df = chr_df.reset_index(drop=True)

In [ ]:
chr_df

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_058066.1,RefSeq,region,1,39361238,.,+,.,ID=NC_058066.1:1..39361238;Dbxref=taxon:45264;...
1,NC_058067.1,RefSeq,region,1,37394699,.,+,.,ID=NC_058067.1:1..37394699;Dbxref=taxon:45264;...
2,NC_058068.1,RefSeq,region,1,19500522,.,+,.,ID=NC_058068.1:1..19500522;Dbxref=taxon:45264;...
3,NC_058069.1,RefSeq,region,1,29333776,.,+,.,ID=NC_058069.1:1..29333776;Dbxref=taxon:45264;...
4,NC_058070.1,RefSeq,region,1,31085176,.,+,.,ID=NC_058070.1:1..31085176;Dbxref=taxon:45264;...
5,NC_058071.1,RefSeq,region,1,19840543,.,+,.,ID=NC_058071.1:1..19840543;Dbxref=taxon:45264;...
6,NC_058072.1,RefSeq,region,1,24214911,.,+,.,ID=NC_058072.1:1..24214911;Dbxref=taxon:45264;...
7,NC_058073.1,RefSeq,region,1,17823322,.,+,.,ID=NC_058073.1:1..17823322;Dbxref=taxon:45264;...
8,NC_058074.1,RefSeq,region,1,19706214,.,+,.,ID=NC_058074.1:1..19706214;Dbxref=taxon:45264;...
9,NC_058075.1,RefSeq,region,1,21761158,.,+,.,ID=NC_058075.1:1..21761158;Dbxref=taxon:45264;...


In [ ]:
annotation_df[(annotation_df['type']=='region') & (annotation_df['seqid'].isin(chr_seqid))]['end'].sum()

np.int64(339964128)

In [ ]:
annotation_df[(annotation_df['type']=='region')]['end'].sum()

np.int64(475381253)

In [8]:
def add_introns(annotation_df):
    introns = []
    for gene in annotation_df[annotation_df['type'] == 'gene'].itertuples():
        gene_exons = annotation_df[(annotation_df['type'] == 'exon') &
                                   (annotation_df['seqid'] == gene.seqid) &
                                   (annotation_df['start'] >= gene.start) &
                                   (annotation_df['end'] <= gene.end)]
        gene_exons = gene_exons.sort_values(by='start')
        previous_exon_end = None
        for exon in gene_exons.itertuples():
            if previous_exon_end is not None:
                intron_start = previous_exon_end + 1
                intron_end = exon.start - 1
                if intron_start < intron_end:
                    introns.append({
                        "seqid": gene.seqid,
                        "source": "predicted",
                        "type": "intron",
                        "start": intron_start,
                        "end": intron_end,
                        "score": ".",
                        "strand": gene.strand,
                        "phase": ".",
                        "attributes": gene.attributes
                    })
            previous_exon_end = exon.end

    introns_df = pd.DataFrame(introns)
    return pd.concat([annotation_df, introns_df], ignore_index=True)

annotation_intron_df = add_introns(annotation_df)

In [9]:
annotation_intron_df = annotation_intron_df[annotation_intron_df['seqid'].isin(chr_seqid)].sort_values(['seqid', 'start'], ascending=[True, True])


In [ ]:
annotation_intron_df

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_058066.1,RefSeq,region,1,39361238,.,+,.,ID=NC_058066.1:1..39361238;Dbxref=taxon:45264;...
1,NC_058066.1,Gnomon,gene,1962,23221,.,-,.,ID=gene-LOC114963522;Dbxref=GeneID:114963522;N...
2,NC_058066.1,Gnomon,lnc_RNA,1962,23221,.,-,.,ID=rna-XR_003825913.2;Parent=gene-LOC114963522...
6,NC_058066.1,Gnomon,exon,1962,2119,.,-,.,ID=exon-XR_003825913.2-4;Parent=rna-XR_0038259...
826501,NC_058066.1,predicted,intron,2120,15360,.,-,.,ID=gene-LOC114963522;Dbxref=GeneID:114963522;N...
...,...,...,...,...,...,...,...,...,...
682963,NC_058079.1,Gnomon,exon,17420715,17420774,.,+,.,ID=exon-XM_044307872.1-26;Parent=rna-XM_044307...
682989,NC_058079.1,Gnomon,CDS,17420715,17420774,.,+,0,ID=cds-XP_044163807.1;Parent=rna-XM_044307872....
997594,NC_058079.1,predicted,intron,17420775,17420892,.,+,.,ID=gene-LOC114952830;Dbxref=GeneID:114952830;N...
682964,NC_058079.1,Gnomon,exon,17420893,17421004,.,+,.,ID=exon-XM_044307872.1-27;Parent=rna-XM_044307...


In [10]:
def classify_structure(structure, annotation_df):
    seqid = structure['seqid']
    start = structure['start']
    end = structure['end']
    length = end - start + 1
    midpoint = start + length // 2

    chromosome_annotations = annotation_df[annotation_df['seqid'] == seqid]
    filtered_annotations = chromosome_annotations[
        ((chromosome_annotations['start'] - 1000 <= end) & (chromosome_annotations['end'] + 1000 >= start))
    ]

    promoter_range = 1000
    downstream_range = 200

    for _, row in filtered_annotations.iterrows():
        if row['type'] == 'gene':
            tss = row['start'] if row['strand'] == '+' else row['end']
            tes = row['end'] if row['strand'] == '+' else row['start']

            if (row['strand'] == '+' and tss - promoter_range <= midpoint <= tss) or \
               (row['strand'] == '-' and tss <= midpoint <= tss + promoter_range):
                return "promoter"

            if (row['strand'] == '+' and tes <= midpoint <= tes + downstream_range) or \
               (row['strand'] == '-' and tes - downstream_range <= midpoint <= tes):
                return "downstream"

        else:
          if row['start'] <= midpoint <= row['end']:
              return row['type']

    return "intergenic"